# Set up workspace parameters

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: workspace_perso
Azure region: westeurope
Subscription id: 666cdcbd-0cde-4fa3-9162-4eafb2b8edbe
Resource group: ressource1


# Set up Compute cluster 

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name = "udacity-project"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            vm_priority='lowpriority',
                                                            max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)


# Set up HyperDrive configuration and runs

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice(25,50,75,100,125,150,175,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, slack_amount=None, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
# Note that conda isn't the only spec available, you can set up compute env with pip or a dockerfile.
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',                       
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=10,
                                max_concurrent_runs=4)

## Run HyperDrive expriments

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0
Web View: https://ml.azure.com/runs/HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0?wsid=/subscriptions/666cdcbd-0cde-4fa3-9162-4eafb2b8edbe/resourcegroups/ressource1/workspaces/workspace_perso&tid=daf12e72-6490-4f70-a448-7ea21423f8b8

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-05T14:00:24.550355][API][INFO]Experiment created<END>\n""<START>[2022-04-05T14:00:25.692828][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-04-05T14:00:26.352206][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0
Web View: https://ml.azure.com/runs/HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0?wsid=/subscriptions/666cdcbd-0cde-4fa3-9162-4eafb2b8edbe/resourcegroups/ressource1/workspaces/workspace_perso&tid=daf12e72-6490-4f70-a448-7ea21423f8b8



{'runId': 'HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2022-04-05T14:00:24.315351Z',
 'endTimeUtc': '2022-04-05T14:16:49.005718Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '27ed7ab3-01d8-44b8-b56c-ba8fcb0a93cb',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': 'infinite_space_size',
  'score': '0.91350531107739',
  'best_child_run_id': 'HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0_9',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://workspaceperso5448820782.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_bf0c2c3d-713b-4

## Get results and save the best one

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f'Best Run Id: {best_run.id}')
print(f'Best run parameters : {parameter_values}')
print(f'Accuracy: {best_run_metrics["Accuracy"]:.3f}')

Best Run Id: HD_bf0c2c3d-713b-44ce-ac46-80f90e50feb0_9
Best run parameters : ['--C', '0.20624530875187946', '--max_iter', '125']
Accuracy: 0.914


In [7]:
best_run.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'logs/azureml/dataprep/rslex.log.2022-04-05-14',
 'outputs/.amlignore',
 'outputs/model.joblib',
 'outputs/model_0.20624530875187946_125.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [8]:
joblib.dump(value="model.joblib", filename="outputs/model.joblib")

['outputs/model.joblib']

# Set up AutoML run

## Refactor datatset

In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
csv_url= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=csv_url)

In [33]:
ds

{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [34]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [35]:
# WARNING:root:The AutoMLConfig parameters, X and y, will soon be deprecated. Please refer to our documentation for the latest interface: https://aka.ms/AutoMLConfig
# will have to concat x and y in the future and "use training_data" and "label_column_name" parameters

# Reference: "Create a dataset from pandas dataframe" section 
# at https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets

import pandas as pd
from azureml.core import Dataset, Datastore

training_data = pd.concat([x, y], axis = 1)

training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [36]:
#dirname = "./training_data"
os.makedirs('training_data', exist_ok=True)

local_path = './training_data/training_data.csv'
training_data.to_csv(local_path, index=False)

In [37]:
Dataset.File.upload_directory(src_dir= "training_data", target=(datastore, "training_data"), overwrite=True)
ds = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('training_data/training_data.csv'))])

Validating arguments.
Arguments validated.
Uploading file to training_data
Uploading an estimated of 3 files
Uploading training_data/.amlignore
Uploaded training_data/.amlignore, 1 files out of an estimated total of 3
Uploading training_data/.amlignore.amltmp
Uploaded training_data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading training_data/training_data.csv
Uploaded training_data/training_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files
Creating new dataset


In [38]:
ds

{
  "source": [
    "('workspaceblobstore', 'training_data/training_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [39]:
ds = ds.to_pandas_dataframe()

In [40]:
ds.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


## Register dataset

In [10]:
# get the datastore to upload prepared data
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-dataset-from-pandas-dataframe
datastore = ws.get_default_datastore()
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-2b053715-edc8-4d3b-a8c1-d7ee4050384e",
  "account_name": "workspaceperso5448820782",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [11]:
dataset = Dataset.Tabular.register_pandas_dataframe(training_data, datastore, "udacity_project_dataset", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/68b2d79b-b846-41db-ba7a-e11e790678ba/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## Set up AutoML experiment

In [31]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
                             experiment_timeout_minutes=30,
                             task="classification",
                             training_data = ds,
                             label_column_name = "y",
                             compute_target=cpu_cluster,
                             iterations=30,
                             iteration_timeout_minutes=5,
                             primary_metric="accuracy",
                             n_cross_validations=5
                            )


## Run AutoML

In [32]:
# Submit your automl run

automl_run = exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b07a763b-8e10-44da-925e-b794e0fe356c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b07a763b-8e10-44da-925e-b794e0fe356c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Generating features for the dataset.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+-----------------------------

{'runId': 'AutoML_b07a763b-8e10-44da-925e-b794e0fe356c',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2022-04-05T14:26:26.076503Z',
 'endTimeUtc': '2022-04-05T15:08:31.871025Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity-project',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"666cdcbd-0cde-4fa3-9162-4eafb2b8edbe","resource_group":"ressource1","workspace_name":"workspace_perso","region":"westeurope","compute_target":"udacity-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_

## Get results and save the best one

In [33]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_b07a763b-8e10-44da-925e-b794e0fe356c_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=None, reg_alpha=0.9473684210526315, reg_lambda=0.42105263157894735, subsample=0.49526315789473685))], verbose=False)), ('15', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features='sqrt', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, 

In [34]:
run_properties = best_run.properties
run_properties["model_output_path"]

'outputs/model.pkl'

In [35]:
run_properties["score"]

'0.9180880121396056'

In [36]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = 'outputs/')

Model(workspace=Workspace.create(name='workspace_perso', subscription_id='666cdcbd-0cde-4fa3-9162-4eafb2b8edbe', resource_group='ressource1'), name=automl_best_model.pkl, id=automl_best_model.pkl:4, version=4, tags={}, properties={})

## Compute cluster cleaning

In [ ]:
cpu_cluster.delete()